Step 1: Import sqlite3 and Create a Database

In [2]:
import sqlite3 

# Create a connection to a database file (or use ':memory:' for temporary DB)

conn = sqlite3.connect('weather_data.db') # This creates weather_data.db in your folder

cursor = conn.cursor()
print("Connected to:", conn.execute("PRAGMA database_list").fetchall())

Connected to: [(0, 'main', 'J:\\Asad-Learning\\Python-Learning\\weather_data.db')]


In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())


[('sqlite_sequence',), ('employee',), ('weather',)]


.

🪜 Step 2: Create a Table
Let’s create a simple table for weather data:



In [72]:
create_table_query = '''
        CREATE TABLE IF NOT EXISTS weather(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            city TEXT,
            state TEXT,
            tempereture REAL,
            max_temp REAL,
            min_temp REAL,
            wind_speed REAL,
            precipitation REAL
            );
'''
cursor.execute(create_table_query)
conn.commit()

In [73]:
cursor.execute('PRAGMA table_info(weather)')
for col in cursor.fetchall():
    print(col)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'date', 'TEXT', 0, None, 0)
(2, 'city', 'TEXT', 0, None, 0)
(3, 'state', 'TEXT', 0, None, 0)
(4, 'tempereture', 'REAL', 0, None, 0)
(5, 'max_temp', 'REAL', 0, None, 0)
(6, 'min_temp', 'REAL', 0, None, 0)
(7, 'wind_speed', 'REAL', 0, None, 0)
(8, 'precipitation', 'REAL', 0, None, 0)


In [21]:
# cursor.execute("ALTER TABLE weather RENAME COLUMN tempreture TO temperature")
# conn.commit()


🪜 Step 3: Insert Sample Data
Here’s how you insert one row manually:

In [74]:
create_data_query = '''
    INSERT INTO weather(date, city, state, tempereture, max_temp, min_temp, wind_speed, precipitation)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
'''

values = ('2025-07-29', 'New York', 'NY', 29.5, 32.0, 27.0, 12.5, 0.0)

cursor.execute(create_data_query, values)
conn.commit()


🪜 Step 4: Read Data from the Table

In [75]:
cursor.execute('SELECT * FROM weather')
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, '2025-07-29', 'New York', 'NY', 29.5, 32.0, 27.0, 12.5, 0.0)


Or better — with Pandas:

In [76]:
import pandas as pd 

df = pd.read_sql_query('SELECT * FROM weather', conn)
df

,id,date,city,state,tempereture,max_temp,min_temp,wind_speed,precipitation
0,1,2025-07-29,New York,NY,29.5,32.0,27.0,12.5,0.0


🪜 Step 5: Close the Connection

In [77]:
conn.close()

✅ Project: Import CSV into SQLite & Query It
     Step 1: Load CSV with Pandas

In [45]:
import pandas as pd 

df = pd.read_csv('data.csv')
df.head()



,Name,Age,City,Salary
0,David,34,San Diego,79076
1,Charlie,35,Phoenix,76699
2,Quinn,38,New York,93478
3,Eve,32,Phoenix,92233
4,Paul,27,Houston,96396


Step 2: Create SQLite DB & Table 


if_exists='replace': recreates the table every time

index=False: avoids saving DataFrame index as a DB column

In [ ]:
# import sqlite3

# with sqlite3.connect('mydb.sqlite') as conn:
#     df = pd.read_sql_query("SELECT * FROM table_name", conn)
# # Connection is automatically closed here


In [78]:
import sqlite3 


# Connect to a new or existing SQLite database
conn = sqlite3.connect('weather_data.db')


# Store CSV data into a table named "weather"
df.to_sql('weather', conn, if_exists='replace', index=False)



1

🪜 Step 3: Run SQL Queries with Pandas

In [41]:
# Example: Get group of city and age in asc 
query = '''
SELECT city, Age
FROM weather
GROUP BY city
ORDER BY Age DESC
'''

result_df = pd.read_sql_query(query, conn)
result_df


,City,Age
0,New York,38
1,Los Angeles,36
2,Phoenix,35
3,San Diego,34
4,Chicago,32
5,San Antonio,29
6,Houston,27


You can also try:

In [43]:
pd.read_sql_query( "SELECT * FROM weather WHERE city='New York'", conn)


,Name,Age,City,Salary
0,Quinn,38,New York,93478
1,Hannah,58,New York,59351
2,Hannah,56,New York,84126
3,Steve,38,New York,94769


 Step 4: Close the Connection

In [44]:
conn.close()

✅ Step 1: Setup
🔧 Install SQLAlchemy

In [ ]:
pip install SQLAlchemy


In [4]:
from sqlalchemy import create_engine
# SQLite connection (file-based)
engine = create_engine("sqlite:///products.db")

✅ Step 3: Define Tables Using ORM (Product & Sale)

In [4]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    price = Column(Float)

    # ✅ back_populates points to Sale.product
    sales = relationship("Sale", back_populates='product')

class Sale(Base):
    __tablename__ = 'sales'
    id = Column(Integer, primary_key=True)
    product_id = Column(Integer, ForeignKey('products.id'))
    quantity = Column(Integer)

    # ✅ Use class name "Product", not table name
    product = relationship('Product', back_populates="sales")


Create tables in the database:



In [24]:
Base.metadata.create_all(engine)


In [28]:
from sqlalchemy.orm import Session

# Clear existing data (only for testing/demo!)
session.query(Sale).delete()
session.query(Product).delete()
session.commit()

✅ Step 4: Insert Data

In [25]:

session = Session(bind=engine)

# Add products
p1 = Product(name="Laptop", price="70000")
p2 = Product(name="Mobile", price="45000")

session.add_all([p1,p2])
session.commit()

# Add sales
s1 = Sale(product_id=p1.id, quantity=3)
s2 = Sale(product_id=p2.id, quantity=5)
session.add_all([s1, s2])
session.commit()



✅ Step 5: Query Data

In [27]:
# Query with JOIN
results = session.query(Product.name, Sale.quantity).join(Sale).all()
# …shows all the Sale records, even from previous runs.
for name,qty in results:
    print(f" {name} - {qty} sold")

✅ Step 6: Integrate with Pandas

In [19]:
import pandas as pd 

df = pd.read_sql("SELECT * FROM sales", engine)
df

,id,product_id,quantity
0,1,3,3
1,2,4,5
2,3,5,3
3,4,6,5
4,5,5,2
5,6,6,1
6,7,7,3
7,8,8,5


In [21]:


# Or insert DataFrame into DB
df_new = pd.DataFrame({
    "product_id": [p1.id, p2.id],
    "quantity": [2, 1]
})
df_new.to_sql("sales", con=engine, if_exists="append", index=False)


2

✅ 2. Install Required Python Packages
Run in your terminal or Jupyter Notebook:

In [ ]:
pip install mysql-connector-python sqlalchemy pandas


✅ 3. Connect Python to MySQL
🔹 Using mysql-connector:

In [6]:
import mysql.connector

print("🚀 Starting MySQL connection test...")
print("⏳ Trying to connect...")

try:
    print("🛠 Attempting to connect with parameters...")
    conn = mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='*Notebook@2025',
        database='30day_sql_query',
        connection_timeout=5
    )
    print("✅ Connected successfully")
    conn.close()
except mysql.connector.Error as err:
    print("❌ Connection failed:")
    print(err)


🚀 Starting MySQL connection test...
⏳ Trying to connect...
🛠 Attempting to connect with parameters...
✅ Connected successfully


In [28]:
import mysql.connector

conn = mysql.connector.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    password='*Notebook@2025',
    database='30day_sql_query',
    connection_timeout=5
)

cursor = conn.cursor()


In [29]:
cursor = conn.cursor()

cursor.execute("SHOW TABLES")
for table in cursor:
    print(table)


('brands',)
('day_indicator',)
('deduction',)
('emp_transaction',)
('employee',)
('footer',)
('income',)
('mountain_huts',)
('products',)
('q4_data',)
('salary',)
('student_tests',)
('trails',)


# This way is good 

In [4]:
import pandas as pd
import mysql.connector

df = pd.read_csv("output.csv")

# Step 2: Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='*Notebook@2025',
    database='30day_sql_query'
)
cursor = conn.cursor()



In [2]:
query = "SHOW TABLES"
cursor.execute(query)
tables = cursor.fetchall()
for table in tables:
    print(table)

('brands',)
('day_indicator',)
('deduction',)
('emp_transaction',)
('employee',)
('footer',)
('income',)
('mountain_huts',)
('products',)
('q4_data',)
('salary',)
('student_tests',)
('trails',)


In [3]:
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS employee")

In [39]:
cursor = conn.cursor()
cursor.execute("SHOW TABLES")
for table in cursor.fetchall():
    print(table)

('brands',)
('day_indicator',)
('deduction',)
('emp_transaction',)
('footer',)
('income',)
('mountain_huts',)
('products',)
('q4_data',)
('salary',)
('student_tests',)
('trails',)


In [40]:
# Step 3: Create the table if not exists
cursor = conn.cursor()

create_table_query = ''' 
    CREATE TABLE IF NOT EXISTS employee(
        Name VARCHAR(100),
        Age INT,
        City VARCHAR(100),
        Salary DECIMAL(10,2))
'''
cursor.execute(create_table_query)

In [5]:
# Step 4: Insert data from DataFrame

df = pd.read_csv('data.csv')
cursor = conn.cursor()
insert_query = "INSERT INTO employee(Name, Age, City, Salary) VALUES(%s, %s, %s, %s)"

for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))
conn.commit()    

ProgrammingError: 1146 (42S02): Table '30day_sql_query.employee' doesn't exist

In [16]:
# df = pd.read_csv('data.csv')
# # Step 4: Insert data from DataFrame
# for _, row in df.iterrows():
#     cursor.execute(
#         "INSERT INTO employee (Name, Age, City, Salary) VALUES (%s, %s, %s, %s)",
#         tuple(row)
#     )
# conn.commit()

In [41]:
df = pd.read_csv('data.csv')

df.head()

,Name,Age,City,Salary
0,David,34,San Diego,79076
1,Charlie,35,Phoenix,76699
2,Quinn,38,New York,93478
3,Eve,32,Phoenix,92233
4,Paul,27,Houston,96396


In [67]:
cursor.execute("DESCRIBE employee")
for row in cursor.fetchall():
    print(row)

('Name', b'varchar(100)', 'YES', '', None, '')
('Age', b'int', 'YES', '', None, '')
('City', b'varchar(100)', 'YES', '', None, '')
('Salary', b'decimal(10,2)', 'YES', '', None, '')


In [ ]:
# cursor = conn.cursor()

# describe_query = "DESCRIBE employee" 
# cursor.execute(describe_query)
# rows = cursor.fetchall()
# for row in rows:
#     print(row)